In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!apt-get install -y ffmpeg
!pip install transformers
!pip install datasets
!pip install torchaudio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 50.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Success

In [1]:
!pip install moviepy


In [17]:
from moviepy.editor import VideoFileClip
import os

def extract_audio(video_path, audio_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)

video_path = '/content/drive/MyDrive/Placecom/video1'
audio_path = '/content/drive/MyDrive/Placecom/video1audio.wav'
extract_audio(video_path, audio_path)

MoviePy - Writing audio in /content/drive/MyDrive/Placecom/video1audio.wav


MoviePy - Done.


In [8]:
if os.path.exists(audio_path):
    print("Audio file created successfully")
else:
    print("Failed to create audio file")

Audio file created successfully


In [11]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 15.0 MB/s eta 0:00:00


In [21]:
import speech_recognition as sr

def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    audio_file = sr.AudioFile(file_path)

    with audio_file as source:
        audio_data = recognizer.record(source)

    # Perform the transcription
    try:
        text = recognizer.recognize_google(audio_data)
        return text
    except sr.RequestError:
        return "API unavailable"
    except sr.UnknownValueError:
        return "Unable to recognize speech"

# Path to the WAV file
audio_file_path = '/content/drive/MyDrive/Placecom/video1audio.wav'

# Transcribe the audio
transcription = transcribe_audio(audio_file_path)
print(f"Transcribed Text: {transcription}")


Transcribed Text: good day to everyone this is Nickelback I'm from MSC application and I'm here to give me directions to the place called The I cancel some motivated and a communicate that I'm an ideal candidate for a place called because my communication skills always have enabled me to convey my ideas clearly and efficiently attraction please is my capability I can translate translate this ability ensures that the meetings and discussions and focused and productive leading to better outcomes and more efficiently useful during efficient communication is key to making a positive pressure my ability to articulate the value proportion of Sr clearly and possibly can help us please and I'm looking forward to hearing back from you guys thank you


In [13]:
!pip install transformers scikit-learn


In [22]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to generate BERT embeddings
def generate_bert_embeddings(text):
    # Tokenize text
    inputs = tokenizer(text, return_tensors="pt")
    # Get BERT model output (last hidden states)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token embedding (first token)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Example transcribed text
transcribed_text = transcription

# Generate embeddings for transcribed text
transcribed_embeddings = generate_bert_embeddings(transcribed_text.lower())

# List of keywords related to self-introduction, public speaking, etc.
keyword_list = [
    "self introduction",
    "public speaking",
    "presentation skills",
    "communication",
    "confidence",
    "skills",
    "projects",
    "organizing",
    "problem solving",
    "critical thinking","agile","time management","improvement","learnings","motivation"
]

# Generate embeddings for each keyword
keyword_embeddings = []
for keyword in keyword_list:
    keyword_embeddings.append(generate_bert_embeddings(keyword.lower()))

# Convert to numpy arrays for cosine similarity calculation
transcribed_embeddings = np.array(transcribed_embeddings)
keyword_embeddings = np.array(keyword_embeddings)


In [15]:
# Function to calculate cosine similarity
def calculate_cosine_similarity(embedding1, embedding2):
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]

# Calculate cosine similarity between transcribed text and each keyword
scores = []
for keyword_emb in keyword_embeddings:
    similarity_score = calculate_cosine_similarity(transcribed_embeddings, keyword_emb)
    scores.append(similarity_score)

# Print scores for each keyword
for keyword, score in zip(keyword_list, scores):
    print(f"Keyword: {keyword}, Similarity Score: {score}")


Keyword: self introduction, Similarity Score: 0.7905502915382385
Keyword: public speaking, Similarity Score: 0.809826672077179
Keyword: presentation skills, Similarity Score: 0.790905237197876
Keyword: communication, Similarity Score: 0.790563702583313
Keyword: confidence, Similarity Score: 0.7713545560836792
Keyword: skills, Similarity Score: 0.7840678691864014
Keyword: projects, Similarity Score: 0.7848294377326965
Keyword: organizing, Similarity Score: 0.7836148142814636
Keyword: problem solving, Similarity Score: 0.8044604063034058
Keyword: critical thinking, Similarity Score: 0.786839485168457
Keyword: agile, Similarity Score: 0.7731953859329224
Keyword: time management, Similarity Score: 0.7454635500907898
Keyword: improvement, Similarity Score: 0.7943359613418579
Keyword: learnings, Similarity Score: 0.7829450964927673
Keyword: motivation, Similarity Score: 0.7610202431678772


In [23]:
import numpy as np

# Example scores (replace with your cosine similarity scores)
s = scores  # Example s for keywords

def calculate_overall_score(s):
    # Normalize s to range between 0 and 1
    normalized_scores = [(score - min(s)) / (max(s) - min(s)) for score in s]

    # Map normalized s to 10-point scale
    overall_score = np.mean(normalized_scores) * 10

    return overall_score

# Example usage with the s
overall_score = calculate_overall_score(s)
print(f"Overall Score out of 10: {overall_score:.2f}")


Overall Score out of 10: 5.92


In [ ]:
# video2 = score for overall relevancy is : Overall Score out of 10: 5.92
# video 1 same score